<a href="https://colab.research.google.com/github/Vijayvkb007/knowledge-graph-for-action-understanding/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import BertTokenizer,BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [2]:
sentence = "A batter is out Bowled if his or her wicket is put down by a ball delivered by the bowler"
tokens = tokenizer.tokenize(sentence)
tokens = ['[CLS]'] + tokens + ['[SEP]']
tokens

['[CLS]',
 'a',
 'batter',
 'is',
 'out',
 'bowled',
 'if',
 'his',
 'or',
 'her',
 'wicket',
 'is',
 'put',
 'down',
 'by',
 'a',
 'ball',
 'delivered',
 'by',
 'the',
 'bowler',
 '[SEP]']

In [3]:
T=30
padded_tokens = tokens + ['[PAD]' for _ in range(T-len(tokens))]
print("Padded tokens are \n {} ".format(padded_tokens))
attn_mask = [ 1 if token != '[PAD]' else 0 for token in padded_tokens  ]
print("Attention Mask are \n {} ".format(attn_mask))

Padded tokens are 
 ['[CLS]', 'a', 'batter', 'is', 'out', 'bowled', 'if', 'his', 'or', 'her', 'wicket', 'is', 'put', 'down', 'by', 'a', 'ball', 'delivered', 'by', 'the', 'bowler', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'] 
Attention Mask are 
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0] 


In [4]:
seg_ids = [0 for _ in range(len(padded_tokens))]

In [5]:
sent_ids = tokenizer.convert_tokens_to_ids(padded_tokens)
token_ids = torch.tensor(sent_ids).unsqueeze(0)
attn_mask = torch.tensor(attn_mask).unsqueeze(0)
seg_ids   = torch.tensor(seg_ids).unsqueeze(0)

print(token_ids)
print(attn_mask)
print(seg_ids)

tensor([[  101,  1037, 23801,  2003,  2041, 19831,  2065,  2010,  2030,  2014,
         12937,  2003,  2404,  2091,  2011,  1037,  3608,  5359,  2011,  1996,
         14999,   102,     0,     0,     0,     0,     0,     0,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]])


In [7]:
output = model(token_ids, attention_mask=attn_mask,token_type_ids=seg_ids)
last_hidden_state, pooler_output = output[0], output[1]

print(last_hidden_state.shape) #hidden states of each token
print(pooler_output) #hidden states of [cls] (forward one linear layer and Tanh activation)
print(pooler_output.shape)

torch.Size([1, 30, 768])
tensor([[-8.7973e-01, -5.4067e-01, -9.5899e-01,  8.4479e-01,  8.8190e-01,
         -4.4379e-01,  6.2297e-01,  2.5535e-01, -8.4881e-01, -1.0000e+00,
         -7.6238e-01,  9.4840e-01,  9.8357e-01,  3.9712e-01,  7.1828e-01,
         -6.3112e-01, -6.5653e-01, -6.2104e-01,  3.7587e-01,  4.3378e-01,
          6.8393e-01,  1.0000e+00, -2.9536e-01,  3.8167e-01,  5.8736e-01,
          9.9287e-01, -7.8775e-01,  8.1887e-01,  9.2251e-01,  7.7154e-01,
         -3.6855e-01,  3.9415e-01, -9.8996e-01, -2.8496e-01, -9.7318e-01,
         -9.9583e-01,  5.9146e-01, -4.7590e-01, -7.0124e-02, -2.2263e-01,
         -8.2177e-01,  5.9511e-01,  1.0000e+00,  1.3950e-02,  7.2416e-01,
         -4.7792e-01, -1.0000e+00,  4.9357e-01, -8.3956e-01,  9.5826e-01,
          9.0536e-01,  9.7730e-01,  2.8484e-01,  6.1069e-01,  6.7739e-01,
         -6.8814e-01,  2.6973e-01,  3.4535e-01, -4.8592e-01, -7.3366e-01,
         -6.5511e-01,  4.2963e-01, -9.0960e-01, -8.7011e-01,  9.6928e-01,
          8.8